In [ ]:
from collections import Counter
from keras.models import load_model
import nltk
import numpy as np
import re
# from google.colab import drive
import os
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model = load_model('model_0.9900.h5')
NGRAM = 4
MAXLEN = 31

alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
# alphabet

In [ ]:
# So a 5-grams contain at most 7*5 = 35 character (except one that has spell mistake)
# add "\x00" padding at the end of 5-grams in order to equal their length
def _encoder_data(text, maxlen=MAXLEN):
        # text = "\x00" + text
        x = np.zeros((maxlen, len(alphabet)))
        for i, c in enumerate(text[:maxlen]):
            x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
          for j in range(i+1, maxlen):
            x[j, 0] = 1
        return x
      
def _decoder_data(x):
    x = x.argmax(axis=-1)
    return ''.join(alphabet[i] for i in x)

In [ ]:
print(_encoder_data("Tôi tên là việt hoàng").shape)
print(_decoder_data(_encoder_data("Tôi tên là Việt Hoàng")))

(31, 199)
Tôi tên là Việt Hoàng          


In [ ]:
def _nltk_ngrams(sentence, n, maxlen):
  list_ngrams = []
  list_words = sentence.split()
  num_words = len(list_words)
  if num_words >= n:
    for ngram in nltk.ngrams(list_words, n):
      if len(' '.join(ngram)) <= maxlen:
        list_ngrams.append(ngram)
  else:
    list_ngrams.append(tuple(list_words)) 
  return list_ngrams

In [ ]:
_nltk_ngrams('xuwr lys ngoon ngữ tự nhieen', NGRAM, MAXLEN)

[('xuwr', 'lys', 'ngoon', 'ngữ'),
 ('lys', 'ngoon', 'ngữ', 'tự'),
 ('ngoon', 'ngữ', 'tự', 'nhieen')]

In [ ]:
def _guess(ngram):
  text = ' '.join(ngram)
  preds = model.predict(np.array([_encoder_data(text)]))
  return _decoder_data(preds[0]).strip('\x00')

In [ ]:
_guess(('Phajm', 'vawn', 'ddoongf', 'nguwx', 'tuwj'))

'Phạm văn đồngg nnưư  tưc'

In [ ]:
def _add_punctuation(text, corrected_text):
  list_punctuation = {}
  for (i, word) in enumerate(text.split()):
    if word[0] not in alphabet or word[-1] not in alphabet:
      start_punc = ''
      for c in word:
        if c in alphabet:
          break
        start_punc += c
      end_punc = ''
      for c in word[::-1]:
        if c in alphabet:
          break
        end_punc += c
      end_punc = end_punc[::-1]
      list_punctuation[i] = [start_punc, end_punc]
  result = ''
  for (i, word) in enumerate(corrected_text.split()):
    if i in list_punctuation:
      result += (list_punctuation[i][0] + word + list_punctuation[i][1]) + ' '
    else:
      result += word + ' '
  return result.strip()

In [ ]:
def _correct(text):
  new_text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)
  # print('new_text: ', new_text)
  ngrams = list(_nltk_ngrams(new_text, NGRAM, MAXLEN))
  # print('ngrams: ', ngrams)
  new_guessed_ngrams = list(_guess(ngram) for ngram in ngrams)
  guessed_ngrams = []
  for ele in new_guessed_ngrams: 
    x = ele.replace('  ', ' ')
    guessed_ngrams.append(x)
  # print('guessed_ngrams: ', guessed_ngrams)
  candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
  # print('candidates: ', candidates)
  for nid, ngram in (enumerate(guessed_ngrams)):
    # print('nid: ', nid, ' ----- ngram: ', ngram)
    for wid, word in (enumerate(re.split('\s', ngram))):
      # print('---> wid: ', wid, ' ----- word: ', word)
      candidates[nid + wid].update([word])
      # print('---> candidates_update: ', candidates[nid + wid].update([word]))
  corrected_text = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
  print('text: ', text)
  print('corrected_text: ', corrected_text)
  return _add_punctuation(text, corrected_text)

In [ ]:
text = ['25 phan vawn ddajt, phuowfng beesn nghes, quaanj 1, thafnh phoos hoof chi minh',
        '234 phajm vawn ddoofg, phuowfng linh trung, quaajn 2, thafnh phoos hoof chis minh',
        '23 hoaf bifnh phuowngf 3 quaanj 4'
      ]
# text = '25 phan vawn ddajt'
# _correct(text)
for ele in text:
  _correct(ele)

text:  25 phan vawn ddajt, phuowfng beesn nghes, quaanj 1, thafnh phoos hoof chi minh
corrected_text:  25 phân văn đạt phường bến nghe quân 1 thành phố hộ hhi mình
text:  234 phajm vawn ddoofg, phuowfng linh trung, quaajn 2, thafnh phoos hoof chis minh
corrected_text:  234 phạm văn đồng pphượng linh trung quân 2 thành phố hộ khí mình
text:  23 hoaf bifnh phuowngf 3 quaanj 4
corrected_text:  23 hoà bình phương 3 quân 4
